# Tropical Cyclones
## Introduction
 
In this session we will consider the threat posed by *tropical cyclones* -- the biggest beast of them all when it comes to weather hazards. We will be assessing the risk they currently pose (or, rather, have historically posed). 

The use of the term *risk* there was not accidental. We will be going beyond simpy assessing the hazard; we will be estimating impact as measured through total finanical losses (normalised to 2018 USD [$] to account for inflation). 

To work through these important concepts, we will be following a hypothetical scenario, outlined below, in which the US Treasury have hired you and your team as experts in tropical cyclone risk. 

Note that, as last week, some questions are included in the text; the answers to these are at the end of the notebook (and may also be disclosed in the text). 

### Scenario

Your task is to summarise the potential financial impact of tropical cyclones over the next 100 years. They have asked you to assess: 

[1] The expected annual cost of tropical cyclones to the USA, assuming no further climate change. 

[2] The cost of a very expensive (i.e., rare) year -- again in a stationary climate. The Treasury define this as a year with costs exceeded in no more than 1 % of years: the one-in-100-year cost. 

### Data

You will be working estimates of tropical cyclone damage from [ICAT](http://www.icatdamageestimator.com/viewdata). All costs are provided in US Dollars, adjusted for inflation to 2018 equaivalent.   

### Analysis

Follow the workbook to address the requests of the Treasury. Remember to run code cells when you get to them by either pressing "run" at the of the browser window, or by hitting "ctrl" + "enter" on your keyboard. 

The first code cell (below) sets the notebook up for analysis. It also summarises the economic impacts from past hurricanes, printing the mean losses per year and per hurricane. The top ten years (by cost) are printed, and the most costly year is identified. Note that the values here are simple descriptive statistics. 

Run this first code cell now. 

In [ ]:
# Importing and preliminaries
import pandas as pd, numpy as np, matplotlib.pyplot as plt
from random import randint
from scipy import stats
from IPython.display import YouTubeVideo
import matplotlib.pyplot as plt

_nsim=10000
data=pd.read_csv("Data/HurricaneDamage.csv",index_col="lf_ISO_TIME",parse_dates=True)
data=data.loc[data["ND"]>0] # throw away TCs that caused no damage
data["ND"]/=1e9 # Transform cost to billions
anncost=data["ND"].resample("Y").sum()
annidx=pd.date_range(start="%.0f/01/01"%anncost.index.year[0],end="%.0f/01/01"%anncost.index.year[-1],freq="1y")
anncost=anncost.reindex(annidx)
ntc=len(data)
ny=len(anncost)
maxcost_storm=data.loc[data["ND"]==data["ND"].max()]
maxcost_year=anncost.loc[anncost==anncost.max()]
rate=ntc/ny # this is the rate for our TC occurrence model 

print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("%.0f damaging hurricanes in %.0f years"%(ntc,ny))
print("Rate = %.1f storms/year"%rate)
print("*** Cost summaries (amounts in billion, 2018 $ equivalent)***")
print("Mean annual cost is $%.1f"%anncost.mean())
print("Top ten years: ")
for i in range(10):
    print("\t[%.0f] %.0f ($%.2f)"%(i+1,(anncost.sort_values()[::-1]).index.year[i],
          (anncost.sort_values()[::-1])[i]))
print("Mean cost/storm = $%.2f"%data["ND"].mean())
print("Most costly storm = %.0f ($%.2f)"%(maxcost_storm.index.year[0],maxcost_storm["ND"][0]))
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")

Take a moment to appreciate the scale of the costs we're talking about here. For context, the cost of the Furlough scheme in the UK during the Covid-19 pandemic cost ~£70 billion (i.e., just under $100 billion). 

You may recall the devastating 2005 Hurricane season (in which Hurricane Katrina devestated New Orleans); and maybe 2012 (when Hurricane Sandy struck New York). But perhaps 1926 -- and the Great Miami Hurricane -- is an event you have not yet heard of. Run the code below to find out more. 

In [ ]:
# A video explaining the most impactful hurricane you've never heard of. 
YouTubeVideo('LRAJ7Bc0O5E',width=800, height=500)

## The expected value

- [**Q1**] One way of estimating the expected cost is to compute the *sample* mean -- the 'average' of the annual costs. Using that approach, we already have an estimate of the expectation (in 2018 $) from the summary printed above. What is it?

That was easy! But is it a useful answer? Possibly *not*, because as shown by the histogram below, *very* large values (i.e., expensive years) are *not* that unlikley, and that can make the sample mean -- of a relatively short dataset -- unrepresentative of the *expected* annual cost (i.e., the mean value we would observe if we could observe all *possible* years). 

Run the code to see the distribution of annual costs. 

In [ ]:
# Generate histogram of the annual costs
fig,ax=plt.subplots(1,1)
h=ax.hist(anncost,bins=15,facecolor='green',edgecolor='k',linewidth=1,alpha=0.25)
ax.set_ylabel("Count")
ax.set_xlabel("Cost ($2018 Billion)")
ax.grid()

To put the magnitude of variability into perspective, the maximum annual bill (\\$265 billion) is not too far short of 100 times larger than the median value ($3 billion). If such variability existed in human heights, we should expect to encounter people well over 200 m tall! The possibility of such large "events" can make the mean *very* sensitive to sampling variability. To appreciate this, imagine calculating the mean height of all people in this room -- hoping it's a representative sample of all students on campus -- and then picture how that would change if a ~200 m tall student accidentally (tried) to pop in; your mean would go up -- a lot! 

We would be on a much firmer footing to compute the expected value if we knew the statistical distribution of the *population* (of heights, in this example). Knowing the population distribution is equivalent to being able to measure the heights of *all* students on campus. However, seldom can we measure all "members" of a population, so we instead make an assumption about the *probability distribution* (e.g., the Normal, GEV, etc.,) that represents the population, and use the properties of *that* to compute things like the expected value and, for eample, the probability of values beyond given thresholds (via the *CDF* -- as discussed last session).

It would be very helpful, then, to compute the expected value from the probability distribution that represents *annual costs*. We'll get into that soon; first, let's deal with a slightly simpler question: what does the distribution of *costs per cyclone* look like?  


## The distribution of cyclone impacts

The reason why we're breaking things down like this is because, when you think on the processes for a little bit, it makes sense to deal with the damage per cyclone *seperately* from the number of cylones per year. The conceptual basis is explained in [Katz (2002)](https://journals.ametsoc.org/view/journals/apme/41/7/1520-0450_2002_041_0754_smohd_2.0.co_2.xml), but one intuitive reason to seperate them is because the annual cost can equal **exactly** zero (a year in which no damaging hurricanes make landfall), *or **any** other amount*, so our final statistical model must be capable of predicting exactly zero damages, *and* any other amount -- both with the right frequency. For this reason (and some others) it turns out separating our statistical approach into models (distributions) that deal with cyclone occurrence (number of storms per year), from those which consider impact (the cost per storm), is a good idea. So, with that explainer out the way, let's get back to assessing the distribution of costs per hurricane. Run the code to take a look. 

In [ ]:
# Checking the fit for a log-normal distribution 
fig,ax=plt.subplots(1,2)
fig.set_size_inches(8,3)
h1=ax.flat[0].hist(anncost,bins=15,facecolor='green',edgecolor='k',linewidth=1.,density=True,alpha=0.25)
# Taking logs of the costs/storm, and then computing the mean and stdv of the logs
lncost=np.log(data["ND"]) # log transform
lnmean=np.mean(lncost) # mean of log-transformed data
lnstd=np.std(lncost) # stdv of log-transformed data
x=np.linspace(np.min(lncost)-1,np.max(lncost)+2,100)
ax.flat[1].plot(x,stats.norm.pdf(x, loc=lnmean,scale=lnstd),color='red')
h2=ax.flat[1].hist(lncost,bins=15,density=True,facecolor='green',edgecolor='k',linewidth=1.,alpha=0.25)
ax.flat[0].grid(); ax.flat[1].grid()
ax.flat[0].set_ylabel("Density")
ax.flat[0].set_xlabel("Cost ($2018 Billion)")
ax.flat[1].set_xlabel("ln{Cost ($2018 Billion)}")


You will see in the plots that the distribution of costs/hurricane (left) looks very similar to the cost/year distribution: it has very strong positive skew, meaning values *much* greater than the median are not that unlikely. 

The ditribution on the right, however, looks very familiar. It seems to be very well described by the Normal distribution, as shown by the way the red line (the Normal pdf) matches the histogram well. The quantity plotted in this right-hand panel is the cost/hurricane series *after it has been log transformed*. 

*[Note that a log transform just means "taking the logarithm" of all values in the dataset; and the logarithm of a number is the exponent that a base must be raised to in order to equal that number. For example, if the base is 10 and the number is 100, the logarithm would be 2 (as 10$^{2}$=100). In our log transform we use the base e, which has a value of ~2.71]*

The log transform has the effect of reducing larger numbers by greater amounts, and it results in our transformed data following a Normal distrubtion. Technically, this means that the underlying data follow what's called a *Log-Normal distribution*. 

The formula for the expected value (E[X]) from the Log-Normal distribution (given on pg. 88 of the [Wilks textbook](https://ebookcentral.proquest.com/lib/kcl/reader.action?docID=689817)) is: 

\begin{aligned}
E[X]=e^{\mu + \frac{\sigma^{2}}{2}}
\end{aligned}

where $\sigma$ and $\mu$ are the mean and standard deviation, respectively, of the *log-transformed* variable.

The equation therefore tells us that to compute the expected value (i.e., the mean) for the cost/hurricane, we simply (a) log transform the data; (b) compute the mean and standard deviation of the transformed data; and (c) plug those values into the equation above. The code below does this. Run it to find out the expected cost per hurricane.

In [ ]:
lncost=np.log(data["ND"]) # Log transform
lnmean=np.mean(lncost) # Computing the mean of the log-transformed data
lnstd=np.std(lncost) # Computing the standard deviation of the log-transformed data
E=np.exp(lnmean+lnstd**2/2.)
print("The expected value is $%.2f billion per hurricane"%E)

- [**Q2**] How does this compare with the sample average computed at the beginning of this notebook?

## The number of events per year

We now have a statistical model the cost per hurricane, but in order to estimate the expected cost per year, and the cost of a very expensive year (i.e., the one-in-200-year event), we need to also consider a statistical model for the *number of hurricanes per year*. 

What does this distribution look like? Run the code below to take a look. 

In [ ]:
# Checking the Poisson distribution
# Compute the frequency with which ncylones/year are observed
x=np.arange(0,10)
of=np.array([np.sum(data.index.year==ii) for ii in anncost.index.year])
off=np.array([np.sum(of==ii) for ii in x])
gen=stats.poisson.rvs(rate,size=_nsim)
sff=np.array([np.sum(gen==ii) for ii in x])/(_nsim/ny)

fig,ax=plt.subplots(1,1)
ax.bar(x,off,color='blue',alpha=0.8,label="Observed")
ax.bar(x,sff,color='red',alpha=0.25,label="Poisson")
ax.set_xticks(x)
ax.set_xlabel("Hurricanes/year")
ax.set_ylabel("Frequency")
ax.legend()

You will again observe a distribution with positive skew, albeit much less pronounced than in the costs dataset. Although it's not a perfect match, you should additionally notice that the *Poisson* distribution is a reasonably good fit to our data. It succeeds in reproducing the basic shape of our histogram. 

As explained by [Katz (2002)](https://journals.ametsoc.org/view/journals/apme/41/7/1520-0450_2002_041_0754_smohd_2.0.co_2.xml), the Poisson distribution is also theoretically suited to describing the annual frequency of damaging hurricanes, so we will persist with its use here. 

Conveniently, the expected value (E[N]) for the Poisson distribution (i.e., the average number of damaging hurricanes per year) is simply the mean *observed* frequency, known as the "rate". We'll denote this quantity $\lambda$. 

- [**Q3**] What is the value for $\lambda$ here? (Hint look at the stats printed out at the top of this notebook)**.

Once we have determined $\lambda$, we can compute the *expected cost per year* (E[Cy]) by combining the expected value for cost/hurricane E[Ch]) with the expected value of the number of hurricanes per year (E[N]):

\begin{aligned}
E[Cy]=\lambda e^{\mu + \frac{\sigma^{2}}{2}}
\end{aligned}

In other words, we just multiply the two expected values together! 

Run the code below to evaluate the expected cost below and print the number to screen.

In [ ]:
Ecy=np.exp(np.mean(lncost)+lnstd**2/2)*rate
print("The expected cost is $%.2f billion/year"%Ecy)

- [**Q4**] How does this compare to the sample average computed at the beginning of this notebook?

## The magnitude of the one-in-100-year event

The answer above addresses the first request of the Treasury. We must now turn our attention to their second question. How severe should we expect the costs of hurricane damage to be in a very *bad* year? 

This is harder to address with simple formuale, so we will approach the problem via a statistical *simulation*, where we generate a very long sequence of "possible years" that are statistically consistent with our observations. We then summarise the plausible scenario via simple descriptive statistics. 

The algorithm to implement this should be quite intuitive. For each year, we will:

[1] Select a random number from the Poisson distribution to represent the number of damaging hurricanes in that year 

[2] Draw a random number from the Log-Normal distribution to represent its damage

[3] Add up the damage from all hurricanes in the year to obtain the total damage. 


If we do this for many, many years (say 100,000), we should get a very good idea of what's likely for the 1-in-100 year event (because we will experience 1000 events as 'extreme' in the 100,000 year series). Computing the 99th percentile from the simulated timeseries provides us with the estimate of the 1-in-100 year event. 

When you have read (and understood) the above, run the code below to generate some plausible scenarios! 

Once the simulations are complete a timeseries plot showing all the plausible years is plotted, and the 99th percentile is printed to screen. For reference, we also highlight (with a red line) the most costly year in the observations (1926). 

In [ ]:
# Stochastic simulation
nsim=100000# This sets the number of years in our simulation
nstorms=np.zeros(nsim,dtype=np.int)
cost=np.zeros(nsim)
for i in range(nsim):
    nstorms[i]=int(stats.poisson.rvs(rate))
    cost[i]=np.sum(np.exp(stats.norm.rvs(loc=lnmean,scale=lnstd,
                                         size=nstorms[i])))
fig,ax=plt.subplots(1,1)
fig.set_size_inches(7,3)
fig.set_dpi(200)
ax.plot(cost)
ax.set_xlabel("Simulated year number")
ax.set_ylabel("Cost ($ Billion)")
ax.axhline(anncost.max(),color='red',label="1926 cost")
ax.set_ylim(0,np.max(cost))
ax.legend()
print(" 99th percentile/1-in-100-year event would be $%.2f billion"%np.percentile(cost,99))

You will all get slightly different answers for the above due the randomness of the simulation, but with a large enough value for nsim, I expect you are unlikely to differ by more than ~20 %. 

- [**Q5**] How reaslistic, do you think the largest events in the simulated series are? If you judge them to be unreaslitic, why?

## Wrap up

We've covered a lot of ground here. It's now time to take stock. 

#### What did we do? 
Under a scenario of responding to a request from the US Treasury, we summarised the *observed* economic impacts from hurricanes making landfall in the USA. We then assessed the statistical distributions of the impact per hurricane (the Log-Normal), and the number of hurricanes per year (Poisson). We used a simple formula to compute the expected hurricane cost per year using paramaters from the the Log-Normal and Poisson distributions. To evaluate the expected magnitude of the 1-in-100-year hurricane season, we then used a "*stochastic simulation*" (i.e., the random sampling from the Poisson and Log-Normal distributions). This is a flexible, widely-used approach to explore risk resulting from potentially complex interacting processes.  

#### Answers 

- [Q1] \$18.9 Billion (2018 adjusted)

- [Q2] It is substantially higher (23.06 vs. 9.8 billion \$ per hurricane)

- [Q3] The value for $\lambda$ is 2.1 storms/year

- [Q4] It is substantially higher (48.29 vs. 18.9 billion \$ per year)

- [Q5] Some points to consider: (i) the total value of all assets may be below some of the highest simulated losses; (ii) there may be a physical limit to the number of tropical cyclones per year; (iii) perhaps the damage/hurricane is *not* independent from the total number of hurricanes 

#### Additional questions
[1] Upon reading the report you prepared for the Treasury, the *US Secretary of Homeland Security* concludes: 

"*Hurricanes are bad news. We know that and we've dealt with them just fine for over 100 years. We remember the awful 2005 season and already spend plenty on reducing vulnerability, so I see no reason for us to rethink anything now."*

How would you respond?

[2] List the factors that you think would *increase* hurricane risk (as measured by expected financial losses) over the rest of the century


#### Further reading

[Blackwell 2015](https://ascelibrary.org/doi/abs/10.1061/%28ASCE%29NH.1527-6996.0000162) 

[Conte and Kelly 2018](https://miami.pure.elsevier.com/en/publications/an-imperfect-storm-fat-tailed-tropical-cyclone-damages-insurance-) 

[Flyvbjerg (2020)](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7533687/)

[Katz 2002](https://journals.ametsoc.org/view/journals/apme/41/7/1520-0450_2002_041_0754_smohd_2.0.co_2.xml) 

[Malamud (2004)](https://iopscience.iop.org/article/10.1088/2058-7058/17/8/35/meta) 

[Cirillo and Taleb (2020)](https://www.nature.com/articles/s41567-020-0921-x#Tab1)
